In [2]:

# Data treatment

import numpy as np # linear algebra
import polars as pl # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import duckdb

# OS and Dotenv

from dotenv import load_dotenv
import os
load_dotenv()  

# Deep Learning

import torch                  
import torch.nn as nn          
import torch.nn.functional as F  
import torch.optim as optim  
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, TensorDataset

# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
import duckdb
import polars as pl
from dotenv import load_dotenv
import os
load_dotenv()  


## Only the datasets

dataset_path = r"C:\Users\lucas\Documents\GitHub\Recommind\data\processed"



dataset_ratings = os.path.join(dataset_path, "Books_rating.csv")
dataset_books = os.path.join(dataset_path, "books_data.csv")


books_data = pl.read_csv(dataset_books)
ratings = pl.read_csv(dataset_ratings)

# Register the DataFrames as DuckDB tables
con = duckdb.connect()
con.register('books_data', books_data)
con.register('ratings', ratings)

query = """
SELECT
  books_data.Title,
  books_data.categories,
  books_data.authors
FROM books_data
JOIN ratings
  ON books_data.Title = ratings.Title;
"""

result = con.execute(query).fetchdf()
print(result)





                                                     Title  \
0        Dr. Richard Marrs' Fertility Book: America's L...   
1        Sisterchicks on the Loose! (Sisterchicks Serie...   
2                                          Vagina Warriors   
3        Fifteen sermons preached before the University...   
4        Prosser, Wade and Scwartz's Torts Cases and Ma...   
...                                                    ...   
2999787                Killing Floor (Jack Reacher, No. 1)   
2999788                Killing Floor (Jack Reacher, No. 1)   
2999789                Killing Floor (Jack Reacher, No. 1)   
2999790                Killing Floor (Jack Reacher, No. 1)   
2999791                Killing Floor (Jack Reacher, No. 1)   

                   categories                           authors  
0        ['Health & Fitness']                 ['Richard Marrs']  
1                 ['Fiction']              ['Robin Jones Gunn']  
2             ['Photography']  ['Eve Ensler', 'Joyce Tenn

In [4]:
import duckdb



con = duckdb.connect("proto.duckdb")


dataset_ratings = os.path.join(dataset_path, "Books_rating.csv")
dataset_books = os.path.join(dataset_path, "books_data.csv")

con.execute(f"""
CREATE OR REPLACE TABLE books AS
SELECT * FROM read_csv_auto('{dataset_books}');
""")

con.execute(f"""
CREATE OR REPLACE TABLE ratings AS
SELECT * FROM read_csv_auto('{dataset_ratings}');
""")

result = con.execute("""
SELECT
  books.Title,
  ratings.Rating
FROM books
JOIN ratings ON books.Title = ratings.Title
""").fetchdf()

#print(result.head())




FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

BinderException: Binder Error: Table "ratings" does not have a column named "Rating"

Candidate bindings: : "Id"

LINE 4:   ratings.Rating
          ^

In [42]:

query = """
SELECT *
FROM books
"""

con.execute(query).fetchdf().shape


(212404, 10)

books = Title, authors, categories
ratings = id, Title, User_id, review/score

In [44]:
query = """
SELECT *
FROM ratings
"""

con.execute(query).fetchdf().shape

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(3000000, 10)

In [46]:
query = """SELECT 
    b.Title, 
    b.authors, 
    b.categories, 
    r.Id, 
    r.User_id, 
    r."review/score",
    b.ratingsCount
FROM books b
JOIN ratings r ON b.Title = r.Title;"""

df_merged = con.execute(query).fetchdf()


In [47]:
df_merged

,Title,authors,categories,Id,User_id,review/score,ratingsCount
0,Never Cry Wolf,['Farley Mowat'],['Animal behavior'],0316586390,A3D5EO8SCPTX9W,5.0,34.0
1,Never Cry Wolf,['Farley Mowat'],['Animal behavior'],0316586390,A24N8O90K345KV,5.0,34.0
2,Never Cry Wolf,['Farley Mowat'],['Animal behavior'],0316586390,A2CF0A2HR4OS22,5.0,34.0
3,Never Cry Wolf,['Farley Mowat'],['Animal behavior'],0316586390,None,5.0,34.0
4,Never Cry Wolf,['Farley Mowat'],['Animal behavior'],0316586390,A1VXA74UYHCT9E,1.0,34.0
...,...,...,...,...,...,...,...
2999787,Perelandra: A Novel,['C.S. Lewis'],['Fiction'],B000HMK1SW,A3P5U20M8V1MEX,5.0,61.0
2999788,Perelandra: A Novel,['C.S. Lewis'],['Fiction'],B000HMK1SW,APD7XINUVG5AL,5.0,61.0
2999789,Perelandra: A Novel,['C.S. Lewis'],['Fiction'],B000HMK1SW,A1X4JO8EJ1U5BR,5.0,61.0
2999790,Perelandra: A Novel,['C.S. Lewis'],['Fiction'],B000HMK1SW,AZ9E3TNAKALCE,5.0,61.0


In [48]:
df_merged.isna().sum()

Title                 0
authors          390634
categories       551290
Id                    0
User_id          561774
review/score          0
ratingsCount    1360694
dtype: int64

I'm going to do here everything that i did before, just refactoring from Pandas to SQL.

"but you are still using pandas"

Yes, I am, but the point of using duckdb to simulate a database is to facilitate. We can't use and it's not realistic focusing only on pandas or polars. 

I'm currently testing Duckdb with pandas because in the previous version of this project i had to use pandas. Polars is not so flexibe right now.

This notebook will test and then I will refactor the code

In [49]:
df_merged['categories'] = df_merged['categories'].fillna('No Category')

df_merged['ratingsCount'] = df_merged['ratingsCount'].fillna(0)


In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
import numpy as np
import joblib

# 0. Preprocessing the strings

df_merged['categories'] = df_merged['categories'].str.replace('[', '', regex=False).str.replace(']', '', regex=False)
df_merged['authors'] = df_merged['authors'].str.replace('[', '', regex=False).str.replace(']', '', regex=False)

# 1. Ordinal Encoder

encoder = joblib.load(r'C:\Users\lucas\Documents\GitHub\Recommind\models\encoding_models\ordinal_encoder.joblib')
encoded = encoder.fit_transform(df_merged[['User_id', 'Id', 'categories', 'authors']].to_numpy())

df_encoded = df_merged.copy()
df_encoded['User_id'] = encoded[:, 0].astype(int)
df_encoded['Id'] = encoded[:, 1].astype(int)
df_encoded['categories'] = encoded[:, 2].astype(int)
df_encoded['authors'] = encoded[:, 3].astype(int)


df_encoded = df_encoded[(df_encoded['User_id'] != -1) & (df_encoded['Id'] != -1)]

n_users = df_encoded['User_id'].max() + 1
n_items = df_encoded['Id'].max() + 1

print(f"n_users = {n_users}, n_items = {n_items}")


n_users = 1008962, n_items = 221989


End!

The purpose of this notebook was testing Duckdb.


